In [ ]:
%reload_ext autoreload
%autoreload 2

from importlib import reload
import os
import json
import logging
import datetime

import requests
import tqdm.notebook as tqdm

import pandas as pd
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt

import zcode.inout as zio
import zcode.math as zmath
import zcode.plot as zplot

import kalepy as kale

import weathervane as wv

In [ ]:
PATH_STATION_DATA = "/Users/lzkelley/Programs/weathervane/data/stations"

FNAME_RAW_HISTORY = "/Users/lzkelley/Programs/weathervane/data/raw/isd-history.csv"
FNAME_STATIONS = "/Users/lzkelley/Programs/weathervane/data/us-stations_filtered.csv"
FNAME_INVENTORY = "/Users/lzkelley/Programs/weathervane/data/raw/isd-inventory.csv"

In [ ]:
def sci_not(val, **kwargs):
    kwargs.setdefault('man', 1)
    kwargs.setdefault('dollar', False)
    kwargs.setdefault('sign', True)
    return zplot.scientific_notation(val, **kwargs)

In [ ]:
reload(wv)
reload(wv.main)
reload(wv)
stations = wv.Stations()
# stat = stations.fuzz("san francisco", print_top=True)
stat, *_ = stations.nearest('santa barbara')
print(stat)

# Examine Data

In [ ]:
data = wv.load_station_data(stat)
data.keys()

In [ ]:
", ".join(list(data.keys()))

In [ ]:
# data['AA1'].loc[~data['AA2'].isnull()]

KEY = 'GD1'

# data[KEY].loc[[(str(ss) != 'nan') and (ss[2:4] != '99') for ss in data[KEY]]]

# print(data[KEY][693000:])
# print(data[~data[KEY].isnull()]['DATE'])
print(zmath.frac_str(~data[KEY].isnull()))

# print(data[KEY][~data[KEY].isnull()].iloc[220])

In [ ]:

logging.getLogger().setLevel(20)
years, results, gidx = wv.station_to_grid(data);
temp = results['temp']
print(temp.shape, zmath.stats_str(temp))
# noise = np.clip(np.random.normal(scale=0.5, size=temp.shape), -1.5, +1.5)
# temp = sp.constants.convert_temperature(temp, 'Celsius', 'Fahrenheit') + noise


# Cloud Cover

In [ ]:
clouds = results['clouds']

In [ ]:
cave = np.mean(clouds, axis=0)
levels = None
# levels = np.arange(0, 4.1, 0.5)

fig, ax = plt.subplots(figsize=[16, 6])
pcm = ax.pcolormesh(cave.T, cmap='Greys')
cbar = plt.colorbar(pcm, label='Cloud Cover [0-4]')

ax.axhline(12, color='0.5', ls='--')

edges = [np.arange(365), np.arange(24)]
*_, cc = kale.plot.draw_contour2d(
    ax, edges, cave, cmap=pcm.cmap.reversed(),
    levels=levels, smooth=3, upsample=2, pad=2, cbar=cbar
)

ax.set(xlim=[0, 365], xlabel='Day of Year', ylim=[0, 24], ylabel='Hour of Day')
plt.show()

In [ ]:
# levels = [50, 60, 65]
levels = None

nyr, nday, nhr = clouds.shape
day_ave = np.mean(clouds, axis=-1).T
edges = [np.arange(nday), years]

fig, ax = plt.subplots(figsize=[16, 6])
pcm = ax.pcolormesh(*edges, day_ave.T, shading='auto', cmap='Greys')
cbar = plt.colorbar(pcm, label='Ave Cloud Cover [0-4]')

*_, cc = kale.plot.draw_contour2d(
    ax, edges, day_ave,
    levels=levels, smooth=3, upsample=2, pad=2, cbar=cbar
)

ax.set(xlim=[0, nday], xlabel='Day of Year', ylim=zmath.minmax(years), ylabel='Year')
plt.show()

In [ ]:
fig, axes = zplot.figax(ncols=2, scale='lin', sharey=True)
confs = [50, 90]

shp = clouds.shape

def plot_ave_axis(ax, temp, axis):
    test = np.moveaxis(temp, axis, 0).reshape(shp[axis], -1)
    tave = test.mean(axis=-1)
    hh, = ax.plot(tave)

    for pp in confs:
        conf = np.percentile(test, [50-pp/2, 50+pp/2], axis=-1)
        ax.fill_between(np.arange(tave.shape[0]), *conf, alpha=0.2, color=hh.get_color())

    return

plot_ave_axis(axes[0], clouds, 1)
plot_ave_axis(axes[1], clouds, 2)

plt.show()



In [ ]:
confs = [20, 40]

plt.figure(figsize=[12, 4])
plt.grid(alpha=0.2)

test = clouds.reshape(np.shape(temp)[0], -1)
day_ave = np.mean(test, axis=-1)
day_med = np.median(test, axis=-1)
xx = years

ave_coeff, ave_fit = zmath.numeric.regress(xx, day_ave)

hh, = plt.plot(xx, day_ave, ls=':', label='ave')
col = hh.get_color()
plt.plot(xx, day_med, color=col, ls='--', label='med')
plt.plot(xx, ave_fit, color=col, ls='-', label='ave fit')
plt.title(fr'${sci_not(ave_coeff[0])} \; \mathrm{{\#/yr}}$')

for pp in confs:
    conf = np.percentile(test, [50-pp/2, 50+pp/2], axis=-1)
    plt.fill_between(xx, *conf, alpha=0.2, label=fr'${pp}\%$', color=col)

plt.legend()
plt.show()

In [ ]:
xx = years[:, np.newaxis, np.newaxis] * np.ones_like(clouds)

coeff, zz = zmath.numeric.regress(xx, clouds)
slope = coeff[0]

levels = None
# levels = zmath.spacing(tave/10, scale='lin', integers=True) * 10
# print(levels)
# levels = [-0.01, 0.01]

smap = zplot.smap(slope, cmap='RdBu_r', midpoint=0.0)
cmap = smap.cmap
# cmap = 'RdBu_r'

fig, ax = plt.subplots(figsize=[16, 6])
pcm = ax.pcolormesh(slope.T, cmap=smap.cmap, norm=smap.norm)
cbar = plt.colorbar(pcm, label=r'$\Delta$ Clouds [\#/yr]')

ax.axhline(12, color='0.5', ls='--')

edges = [np.arange(365), np.arange(24)]
*_, cc = kale.plot.draw_contour2d(
    ax, edges, slope, cmap=pcm.cmap.reversed(),
    levels=levels, smooth=3, upsample=2, pad=2, cbar=cbar
)

ax.set(xlim=[0, 365], xlabel='Day of Year', ylim=[0, 24], ylabel='Hour of Day')
plt.show()

# Temperature

In [ ]:
def dump_arrays1d_csv(fname, fmt='g', **kwargs):
    fmt = f"{{:{fmt}}}"
    
    size = None
    header = "#"
    for kk, vv in kwargs.items():
        if (np.ndim(vv) != 1) or ((size is not None) and (vv.size != size)):
            err = f"{kk}.shape = {np.shape(vv)} ({size=})"
            raise ValueError(err)
        size = vv.size
        header += f" {kk}=({vv.size},)"
    
    with open(fname, 'w') as out:
        out.write(header + "\n")
        for kk, vv in kwargs.items():
            line = ", ".join([fmt.format(xx) for xx in vv])
            out.write(line + "\n")
        
    logging.info(f"Saved {fname}', {zio.get_file_size(fname)}")

    return

def dump_array2d_csv(fname, xx, yy, zz, fmt='g'):
    fmt = fmt.strip(':')
    form = f"{{:{fmt}}}"

    if (np.ndim(xx) != 1) or (np.ndim(yy) != 1) or (np.ndim(zz) != 2):
        raise ValueError("Bad shape!")
    
    with open(fname, 'w') as out:
        out.write(f"# {xx.shape=}, {yy.shape=}, {zz.shape=}\n")
        
        line = ", ".join([form.format(vv) for vv in xx])
        out.write(line + "\n")

        line = ", ".join([form.format(vv) for vv in yy])
        out.write(line + "\n")

        for ii in range(np.shape(zz)[0]):
            out.write(line + "\n")
                
        logging.info(f"Saved '{fname}', {zio.get_file_size(fname)}")

    return

In [ ]:
levels = zmath.spacing(tave/10, scale='lin', integers=True) * 10

ax, temp_hist, temp_cont = wv.plot_hour_vs_day(
    temp, station=stat, clabel='Temperature [F]', levels=None, cmap='RdBu_r'
)

# segs = temp_cont._get_allsegs_and_allkinds()
# segs[0]

ax.set(ylim=[0, 23])
plt.show()

fname = wv.station_id(stat)
fname = f"{fname}_temp-ave_hour-vs-day.json"
fname = os.path.join(wv.PATH_DATA_DERIVED, fname)

dump_array2d_csv(fname, temp_hist['x'][:, 0], temp_hist['y'][0], temp_hist['z'])

In [ ]:
tave = np.mean(temp, axis=0)
levels = zmath.spacing(tave/10, scale='lin', integers=True) * 10
print(levels)

fig, ax = plt.subplots(figsize=[16, 6])
pcm = ax.pcolormesh(tave.T)
cbar = plt.colorbar(pcm, label='Temperature [F]')

ax.axhline(12, color='0.5', ls='--')

edges = [np.arange(365), np.arange(24)]
*_, cc = kale.plot.draw_contour2d(
    ax, edges, tave, cmap=pcm.cmap.reversed(),
    levels=levels, smooth=3, upsample=2, pad=2, cbar=cbar
)

ax.set(xlim=[0, 365], xlabel='Day of Year', ylim=[0, 24], ylabel='Hour of Day')
plt.show()

In [ ]:
# levels = [50, 60, 65]
levels = None

nyr, nday, nhr = temp.shape
td_ave = np.mean(temp, axis=-1).T
edges = [np.arange(nday), years]

fig, ax = plt.subplots(figsize=[16, 6])
pcm = ax.pcolormesh(*edges, td_ave.T, shading='auto')
cbar = plt.colorbar(pcm, label='Ave Temperature [F]')

*_, cc = kale.plot.draw_contour2d(
    ax, edges, td_ave,
    levels=levels, smooth=3, upsample=2, pad=2, cbar=cbar
)

ax.set(xlim=[0, nday], xlabel='Day of Year', ylim=zmath.minmax(years), ylabel='Year')
plt.show()

In [ ]:
reload(wv)

In [ ]:
shp = temp.shape


def data_ave_axis(ax, vals, axis, smooth=None):
    confs = np.arange(0, 11)*10
    test = np.moveaxis(vals, axis, 0).reshape(shp[axis], -1)
    ave = test.mean(axis=-1)
    if smooth is not None:
        sm = smooth * ave.size
        ave = wv.smooth(ave, sm)

    data = dict(x=np.arange(ave.size), ave=ave)
        
    for pp in confs:
        conf = np.percentile(test, pp, axis=-1)
        if smooth is not None:
            conf = wv.smooth(conf, sm)
        data[f'conf{pp:.0f}'] = conf
            
    return data

temp_vs_day = data_ave_axis(axes[0], temp, 1, smooth=0.04)
temp_vs_hr = data_ave_axis(axes[1], temp, 2, smooth=0.04)

path_stat = wv.station_id(stat)
path_stat = os.path.join(wv.PATH_DATA_DERIVED, path_stat, '')
zio.check_path(path_stat)

fname = f"temp-ave_vs-day.csv"
fname = os.path.join(path_stat, fname)
dump_arrays1d_csv(fname, **temp_vs_day)

fname = f"temp-ave_vs-hr.csv"
fname = os.path.join(path_stat, fname)
dump_arrays1d_csv(fname, **temp_vs_hr)


In [ ]:
fig, axes = zplot.figax(ncols=2, scale='lin', sharey=True)
# confs = [50, 90]
confs = np.arange(0, 11)*10

shp = temp.shape

def plot_ave_axis(ax, temp, axis, smooth=None):
    test = np.moveaxis(temp, axis, 0).reshape(shp[axis], -1)
    tave = test.mean(axis=-1)
    if smooth is not None:
        sm = smooth * tave.size
        tave = wv.smooth(tave, sm)
    hh, = ax.plot(tave)

    for pp in confs:
        conf = np.percentile(test, [50-pp/2, 50+pp/2], axis=-1)
        if smooth is not None:
            conf = [wv.smooth(cc, sm) for cc in conf]
        
        ax.fill_between(np.arange(tave.shape[0]), *conf, alpha=0.2, color=hh.get_color())

    return

plot_ave_axis(axes[0], temp, 1, smooth=0.04)
plot_ave_axis(axes[1], temp, 2, smooth=0.04)

plt.show()

In [ ]:
confs = [20, 40]

plt.figure(figsize=[12, 4])
plt.grid(alpha=0.2)

test = temp.reshape(np.shape(temp)[0], -1)
tyr_ave = np.mean(test, axis=-1)
tyr_med = np.median(test, axis=-1)
# xx = np.arange(test.shape[0])
xx = years

ave_coeff, ave_fit = zmath.numeric.regress(xx, tyr_ave)

hh, = plt.plot(xx, tyr_ave, ls=':', label='ave')
col = hh.get_color()
plt.plot(xx, tyr_med, color=col, ls='--', label='med')
plt.plot(xx, ave_fit, color=col, ls='-', label='ave fit')
plt.title(fr'${sci_not(ave_coeff[0])} \; \mathrm{{deg/yr}}$')

for pp in confs:
    conf = np.percentile(test, [50-pp/2, 50+pp/2], axis=-1)
    plt.fill_between(xx, *conf, alpha=0.2, label=fr'${pp}\%$', color=col)

plt.legend()
plt.show()


In [ ]:
temp.shape
xx = years[:, np.newaxis, np.newaxis] * np.ones_like(temp)

coeff, zz = zmath.numeric.regress(xx, temp)
slope = coeff[0]

levels = None
# levels = zmath.spacing(tave/10, scale='lin', integers=True) * 10
# print(levels)
# levels = [-0.01, 0.01]

smap = zplot.smap(slope, cmap='RdBu_r', midpoint=0.0)
cmap = smap.cmap
# cmap = 'RdBu_r'

fig, ax = plt.subplots(figsize=[16, 6])
pcm = ax.pcolormesh(slope.T, cmap=smap.cmap, norm=smap.norm)
cbar = plt.colorbar(pcm, label=r'$\Delta$ Temperature [F/yr]')

ax.axhline(12, color='0.5', ls='--')

edges = [np.arange(365), np.arange(24)]
*_, cc = kale.plot.draw_contour2d(
    ax, edges, slope, cmap=pcm.cmap.reversed(),
    levels=levels, smooth=3, upsample=2, pad=2, cbar=cbar
)

ax.set(xlim=[0, 365], xlabel='Day of Year', ylim=[0, 24], ylabel='Hour of Day')
plt.show()

In [ ]:
print(temp.shape)
tave = np.mean(temp, axis=-1)
xx = years[:, np.newaxis] * np.ones_like(tave)

coeff, zz = zmath.numeric.regress(xx, tave)
slope = coeff[0]
print(slope.shape)

plt.plot(slope)


In [ ]:
fig, axes = zplot.figax(ncols=2, scale='lin', sharey=True)
confs = [50, 90]

shp = temp.shape

def plot_ave_axis(ax, years, temp, axis, smap=None):
    other = 2 if (axis == 1) else 1
    xlab = 'day of year' if (axis == 1) else 'hour of day'
    tave = np.mean(temp, axis=other)

    xx = years[:, np.newaxis] * np.ones_like(tave)
    slope, zz = zmath.numeric.regress(xx, tave)
    slope = slope[0]
    if smap is None:
        smap = zplot.smap(slope, scale='lin', cmap='RdBu_r', midpoint=0.0)
    colors = smap.to_rgba(slope)
    
    test = np.moveaxis(temp, axis, 0).reshape(shp[axis], -1)
    tave = test.mean(axis=-1)
    xx = np.arange(tave.size)

    hh, = ax.plot(xx, tave)
    ax.scatter(xx, tave, color=colors)

    for pp in confs:
        conf = np.percentile(test, [50-pp/2, 50+pp/2], axis=-1)
        ax.fill_between(xx, *conf, alpha=0.2, color=hh.get_color())

    plt.colorbar(smap, ax=ax, orientation='horizontal', label=r'$\Delta T$ [deg/yr]')
    ax.set(xlabel=xlab, ylabel='Temperature [F]')
    
    return smap

plot_ave_axis(axes[0], years, temp, 1)
plot_ave_axis(axes[1], years, temp, 2)

plt.show()
